In [1]:
import numpy as np
import pyarrow as pa
import ray

In [2]:
from uptrain.core.ops_agg import CosineDist, L2Dist, PartitionOp
from uptrain.core.ops_io import DuckdbReader, ParquetWriter

In [3]:
import atexit

atexit.register(ray.shutdown)

<function ray._private.worker.shutdown(_exiting_interpreter: bool = False)>

In [4]:
runtime_env = {"env_vars": {"NUMBA_DISABLE_PERFORMANCE_WARNINGS": "1"}}
ray.init(runtime_env=runtime_env)

2023-03-28 14:24:18,764	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.3.0


(raylet) /Users/ishananand/toolchains/mambaforge/envs/uptrain_env/lib/python3.10/site-packages/ray/dashboard/agent.py:51: DeprecationWarning: There is no current event loop
(raylet)   aiogrpc.init_grpc_aio()


In [7]:
metric_op = PartitionOp(
    columns=["postId", "model_type", "sig_type"],
    agg_ops=dict(
        cos_initial=CosineDist(value_col="embs", seq_col="views", mode="initial"),
        cos_running=CosineDist(value_col="embs", seq_col="views", mode="running"),
        l2_initial=L2Dist(value_col="embs", seq_col="views", mode="initial"),
        l2_running=L2Dist(value_col="embs", seq_col="views", mode="running"),
    ),
)

In [8]:
import pyarrow.parquet

ds = pa.parquet.read_table(
    "/Users/ishananand/repos/pypypy/notebooks/uptrain/2023-03-13-16-30_2023-03-13-16-45.pq"
)

In [9]:
%%time
metric_actor = metric_op.make_actor()
tbl = metric_actor.run.remote(ds)
result = ray.get(tbl)

CPU times: user 147 ms, sys: 715 ms, total: 861 ms
Wall time: 10.5 s


## appendix

In [8]:
len(ds), len(result)

(118987, 118987)

In [17]:
result["partition_index"]

[
  [
    "1000119245_unified_realtime_ffm_video_unified1",
    "1000304345_unified_realtime_ffm_video_unified1",
    "1000449245_unified_realtime_ffm_video_unified1",
    "1000534345_unified_realtime_ffm_video_unified1",
    "1000703345_unified_realtime_ffm_video_unified1",
    ...
    "9999394345_unified_realtime_ffm_video_unified1",
    "9999394345_unified_realtime_ffm_video_unified1",
    "9999394345_unified_realtime_ffm_video_unified1",
    "9999394345_unified_realtime_ffm_video_unified1",
    "9999860345_unified_realtime_ffm_video_unified1"
  ]
]

In [15]:
np.sum(result["cos_initial"].to_numpy() < 1e-12)

28650

In [13]:
arr = result["partition_index"]

In [16]:
len(arr.unique())

28544